In [1]:
import cv2
import numpy as np
import os
import torch
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
model = YOLO("yolov8x-seg.pt")

In [2]:
def save_video_frames(dir):
    # Create the output directory if it doesn't exist
    if not os.path.exists(dir):
        os.makedirs(dir)
    cap = cv2.VideoCapture(dir+".mp4")

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_filename = os.path.join(dir, f"{frame_count}.jpg")
        cv2.imwrite(frame_filename, frame)
        frame_count += 1
    cap.release()

In [3]:
def remove_cars(video_num):
    for i in range(0,1200):
        source = f"{video_num}/{i}.jpg"
        image = cv2.imread(source)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
        results = model.predict(source, conf=0.2, classes=[2,3,5,7])
        if(results[0].masks == None):
            save_path = f"{video_num}/{i}.jpg"
            image_to_save = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(save_path, image_to_save)
            continue 

        for object in results[0].masks.xy:        
            polygons = []
            for point in object:
                x_coords = np.clip(point[0].astype(int), 0, image.shape[1] - 1)
                y_coords = np.clip(point[1].astype(int), 0, image.shape[0] - 1)
                polygons.append([x_coords, y_coords])

            polygons = np.array(polygons)
            if(len(polygons) == 0):
                continue

            binary_mask = np.zeros(image.shape[:2], dtype=np.uint8)
            cv2.fillPoly(binary_mask, pts=[polygons], color=255)
            blurred_mask = cv2.GaussianBlur(binary_mask, (31, 31), 0)

            black_image = np.zeros_like(image)
            image = np.where(blurred_mask[..., np.newaxis] > 0, black_image, image)   

        save_path = f"{video_num}/{i}.jpg"
        image_to_save = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(save_path, image_to_save) 

In [4]:
def remove_front(dir): 
    for i in range(0,1200): #note, video 4 has 1196 frames, named 0-1195
        img_path = dir+"/"+str(i)+".jpg"
        with Image.open(img_path) as img:
            if img.mode != 'RGB':
                img = img.convert('RGB')
            img_array = np.array(img)

        panel_height = img_array.shape[0] // 4

        img_array[-panel_height:] = [0, 0, 0]


        cv2.imwrite(img_path, cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR))

In [8]:
for i in range(0,1):
    save_video_frames(str(i))
    remove_cars(str(i))
    remove_front(str(i))


image 1/1 /Users/ritamsaha/Desktop/whereabouts/calib_challenge/result_labels/0/0.jpg: 512x640 2 cars, 530.2ms
Speed: 2.1ms preprocess, 530.2ms inference, 2.9ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 /Users/ritamsaha/Desktop/whereabouts/calib_challenge/result_labels/0/1.jpg: 512x640 2 cars, 453.0ms
Speed: 1.5ms preprocess, 453.0ms inference, 1.4ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 /Users/ritamsaha/Desktop/whereabouts/calib_challenge/result_labels/0/2.jpg: 512x640 2 cars, 461.3ms
Speed: 1.6ms preprocess, 461.3ms inference, 1.6ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 /Users/ritamsaha/Desktop/whereabouts/calib_challenge/result_labels/0/3.jpg: 512x640 2 cars, 478.4ms
Speed: 1.6ms preprocess, 478.4ms inference, 1.9ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 /Users/ritamsaha/Desktop/whereabouts/calib_challenge/result_labels/0/4.jpg: 512x640 2 cars, 446.8ms
Speed: 1.6ms preprocess, 446.8ms inference, 1.5ms pos